In [1]:
#import library
import cv2
import numpy as np
import os
import json

In [37]:
import cv2
import numpy as np
import os
import random
# def apply_perspective_transform(img, tilt_range=(-15, 15)):
#     # Convert tilt range from degrees to radians
#     tilt_range = [np.deg2rad(tilt) for tilt in tilt_range]

#     # Choose random tilt angles for x and y within the specified range
#     tilt_x = random.uniform(*tilt_range)
#     tilt_y = random.uniform(*tilt_range)

#     # Compute the perspective transform matrix for the tilt
#     rows, cols = img.shape[:2]
#     src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])
#     print('src_point',src_points)

#     # # Calculate offset caused by tilting
#     # offset_x = cols * np.tan(tilt_y)
#     # offset_y = rows * np.tan(tilt_x)

#     # Calculate the destination points for the perspective transformation
#     dst_points = np.float32([
#         [0 + random.uniform(-0.15,0.15)*cols, 0 + random.uniform(-0.15,0.15)*rows], 
#         [cols + random.uniform(-0.15,0.15)*cols, 0 + random.uniform(-0.15,0.15)*rows], 
#         [0 + random.uniform(-0.15,0.15)*cols, rows + random.uniform(-0.15,0.15)*rows], 
#         [cols + random.uniform(-0.15,0.15)*cols, rows + random.uniform(-0.15,0.15)*rows]
#     ])
#     print('dst_point',dst_points)

#     M = cv2.getPerspectiveTransform(src_points, dst_points)

#     # Apply the perspective transform (tilt)
#     img = cv2.warpPerspective(img, M, (cols, rows))

#     return img
def rotate_and_shear_image(im, bk_height, bk_width):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    change from https://stackoverflow.com/a/51109152
    """
    # Randomize the angle, x_shear, y_shear
    size = random.uniform(0.3, 0.5)  
    angle = random.randint(0, 359)
    x_shear = random.uniform(-0.15, 0.15)
    y_shear = random.uniform(-0.15, 0.15)

    #print(im.shape)
    height, width = im.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_im = cv2.getRotationMatrix2D(image_center, angle, 1.)

    rotation_im[0,1] += x_shear
    rotation_im[1,0] += y_shear

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos_x = abs(rotation_im[0,0]) 
    abs_sin_x = abs(rotation_im[0,1])
    abs_cos_y = abs(rotation_im[1,0]) 
    abs_sin_y = abs(rotation_im[1,1])

    # find the new width and height bounds
    bound_w = int(width * abs_cos_x +  height *  abs_sin_x )
    bound_h = int(height * abs_sin_y  + width * abs_cos_y )

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_im[0, 2] += bound_w/2 - image_center[0] - image_center[1] * x_shear
    rotation_im[1, 2] += bound_h/2 - image_center[1] - image_center[0] * y_shear

    board_mode = cv2.BORDER_CONSTANT
    color = (random.randint(230,250), random.randint(230,250), random.randint(230,250))
    
    # rotate image with the new bounds and translated rotation imrix
    im = cv2.warpAffine(im, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=board_mode, borderValue=color)
    
    # Calculate the ratio size of the image, assume background width > height
    # Set document image size to 0.3 - 0.5 of the background image size
    new_height, new_width = im.shape[:2]
    ratio = size * bk_height / new_height
    final_height = int(new_height * ratio)
    final_width = int(new_width * ratio)

    # Resize the image
    im = cv2.resize(im, (final_width, final_height))
    # im = cv2.resize(im, (0.3, 0.5))

    #print(rotated_im.shape)
    return im


def apply_perspective_transform(img, tilt_range=(-0.15, 0.15)):
    # Convert tilt range from degrees to radians
    tilt_range = [np.deg2rad(tilt) for tilt in tilt_range]

    # Compute the perspective transform matrix for the tilt
    rows, cols = img.shape[:2]
    src_points = np.float32([[0, 0], [cols, 0], [0, rows], [cols, rows]])

    # Calculate the destination points for the perspective transformation
    dst_points = np.float32([
        [0  + cols * np.tan(random.uniform(-0.15, 0.15)), 0 + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [cols + cols * np.tan(random.uniform(-0.15, 0.15)), 0 + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [0  + cols * np.tan(random.uniform(-0.15, 0.15)), rows + rows * np.tan(random.uniform(-0.15, 0.15))], 
        [cols + cols * np.tan(random.uniform(-0.15, 0.15)), rows + rows * np.tan(random.uniform(-0.15, 0.15))]
    ])

    M = cv2.getPerspectiveTransform(src_points, dst_points)

    # Apply the perspective transform (tilt)
    warped_img = cv2.warpPerspective(img, M, (cols, rows))

    # # Calculate the bounding box of the transformed image
    # x_min, y_min = np.min(dst_points, axis=0)
    # x_max, y_max = np.max(dst_points, axis=0)

    # # Add a small margin to the size of the transparent image
    # margin = 10
    # transparent_height = int(y_max - y_min) + margin
    # transparent_width = int(x_max - x_min) + margin

    # # Calculate the offset for placing the transformed image in the transparent image
    # offset_x = int(abs(x_min)) + margin // 4
    # offset_y = int(abs(y_min)) + margin // 4

    # # Create the transparent image with fully transparent background
    # transparent_img = np.zeros((transparent_height, transparent_width, 4), dtype=np.uint8)
    # transparent_img[:, :, 3] = 0  # Set alpha channel to 0 for fully transparent background

    # # Copy the transformed image into the transparent image with proper offset
    # transparent_img[offset_y:offset_y+warped_img.shape[0], offset_x:offset_x+warped_img.shape[1], :3] = warped_img
    # transparent_img[offset_y:offset_y+warped_img.shape[0], offset_x:offset_x+warped_img.shape[1], 3] = 0

    return warped_img


def place_on_background(img, background, size_range, angle_range, tilt_range):
    # Resize image
    size = random.uniform(*size_range)
    img = cv2.resize(img, None, fx=size, fy=size)

    bk_height, bk_width = background.shape[:2]
    # Add random tilts
    # img = apply_perspective_transform(img, tilt_range)
    img = rotate_and_shear_image(img, bk_height, bk_width)
    # img = stackoverflow_rotate_and_shear_image(img)

    # Ensure the document image is not larger than the background
    if img.shape[0] > background.shape[0] or img.shape[1] > background.shape[1]:
        print("Resized document image is larger than the background. Skipping...")
        return None, None

    # Get random position on background
    max_x = background.shape[1] - img.shape[1]
    max_y = background.shape[0] - img.shape[0]
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)

    # Overlay the document image on the background
    # Overlay the document image on the background
    background[y:y+img.shape[0], x:x+img.shape[1], :3] = img[:, :, :3]

    return background, (x, y, img.shape[1], img.shape[0])

# def place_on_background(img, background, size_range, angle_range, tilt_range):
#     # Resize image
#     size = random.uniform(*size_range)
#     img = cv2.resize(img, None, fx=size, fy=size)

#     # Add random tilts
#     img = apply_perspective_transform(img, tilt_range)

#     # Rotate the image by a random angle
#     angle = random.randint(*angle_range)
#     height, width = img.shape[:2]

#     # Calculate the new image dimensions
#     radians = np.radians(angle)
#     new_width = int(abs(width * np.cos(radians)) + abs(height * np.sin(radians)))
#     new_height = int(abs(width * np.sin(radians)) + abs(height * np.cos(radians)))

#     # Rotate the image
#     M = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
#     rotated_img = cv2.warpAffine(img, M, (new_width, new_height))

#     # Create a larger transparent image
#     larger_img = np.zeros((rotated_img.shape[0]*2, rotated_img.shape[1]*2, 4), dtype=np.uint8)

#     # Place the rotated image in the center of the larger transparent image
#     start_y = larger_img.shape[0] // 4
#     start_x = larger_img.shape[1] // 4
#     larger_img[start_y:start_y+rotated_img.shape[0], start_x:start_x+rotated_img.shape[1], :3] = rotated_img
#     larger_img[start_y:start_y+rotated_img.shape[0], start_x:start_x+rotated_img.shape[1], 3] = 255

#     # Ensure the larger transparent image is not larger than the background
#     if larger_img.shape[0] > background.shape[0] or larger_img.shape[1] > background.shape[1]:
#         print("Resized document image is larger than the background. Skipping...")
#         return None, None

#     # Get random position on background
#     max_x = background.shape[1] - larger_img.shape[1]
#     max_y = background.shape[0] - larger_img.shape[0]
#     x = random.randint(0, max_x)
#     y = random.randint(0, max_y)

#     # Overlay the larger transparent image on the background
#     for c in range(3):
#         background[y:y+larger_img.shape[0], x:x+larger_img.shape[1], c] = larger_img[:,:,c] * (larger_img[:,:,3]/255) +  background[y:y+larger_img.shape[0], x:x+larger_img.shape[1], c] * (1 - larger_img[:,:,3]/255)

#     return background, (x, y, larger_img.shape[1], larger_img.shape[0])


# Paths to folders
doc_folder = 'cropped_documents'
bg_folder = 'backgrounds'

# Loop over all document images
for doc_name in os.listdir(doc_folder):
    doc_path = os.path.join(doc_folder, doc_name)
    print(f"Reading document image from: {doc_path}")  # Debug print
    doc_img = cv2.imread(doc_path)
    if doc_img is None:
        print(f"Failed to read document image from: {doc_path}")
        continue

    # Loop over all background images
    for bg_name in os.listdir(bg_folder):
        bg_path = os.path.join(bg_folder, bg_name)
        print(f"Reading background image from: {bg_path}")  # Debug print
        bg_img = cv2.imread(bg_path)
        if bg_img is None:
            print(f"Failed to read background image from: {bg_path}")
            continue

        # Place document on background
        img, bbox = place_on_background(doc_img, bg_img, (0.3, 0.5), (0, 359), (-0.15, 0.15))

        # Check if image is not None before saving
        if img is not None:
            # Save image and annotation
            cv2.imwrite('dataset/{}_{}.png'.format(doc_name, bg_name), img)
            with open('annotations/{}_{}.txt'.format(doc_name, bg_name), 'w') as f:
                f.write(','.join(map(str, bbox)))
        else:
            print(f"Skipping saving for {doc_name} on {bg_name} due to size mismatch.")

Reading document image from: cropped_documents\001_jpg.rf.a831d3c36a427ca0073fbad72a1a6d3c.jpg
Reading background image from: backgrounds\-1x-1.jpg
Reading background image from: backgrounds\10-tips-to-create-a-perfect-workspace-at-home-2124.jpg
Reading background image from: backgrounds\flat-lay-desk-concept-with-copy-space_23-2148459658.jpg
Reading background image from: backgrounds\flat-lay-picnic-basket-with-blanket_23-2148234294.jpg
Reading background image from: backgrounds\full
Failed to read background image from: backgrounds\full
Reading background image from: backgrounds\window-background-fplfozjo0w3jnh76.jpg
Reading background image from: backgrounds\woman-hand-entering-password-code-mobile-phone-paying-bill-with-credit-card-bill-desk-home-office-internet-digital-marketing-mobile-banking-online-payment-digital-technology-concept_33829-624.avif
Failed to read background image from: backgrounds\woman-hand-entering-password-code-mobile-phone-paying-bill-with-credit-card-bill-de

In [14]:
import cv2
import numpy as np
import os
import random



def scale_image(img, bk_height, bk_width):
    # Choose a random scale factor within the specified range
    size = random.uniform(0.3, 0.5)  
    ratio = 1

    if bk_height > bk_width:
        ratio = size * bk_width / img.shape[1]
    else:
        ratio = size * bk_height / img.shape[0]

    # Resize the image
    img = cv2.resize(img, None, fx=ratio, fy=ratio)

    return img

def rotate_and_shear_image(im):
    """
    Rotates an image (angle in degrees) and expands image to avoid cropping
    change from https://stackoverflow.com/a/51109152
    """
    # Randomize the angle, x_shear, y_shear

    angle = random.randint(0, 359)
    x_shear = random.uniform(-0.15, 0.15)
    y_shear = random.uniform(-0.15, 0.15)

    #print(im.shape)
    height, width = im.shape[:2] # image shape has 3 dimensions
    image_center = (width/2, height/2) # getRotationMatrix2D needs coordinates in reverse order (width, height) compared to shape

    rotation_im = cv2.getRotationMatrix2D(image_center, angle, 1.)

    rotation_im[0,1] += x_shear
    rotation_im[1,0] += y_shear

    # rotation calculates the cos and sin, taking absolutes of those.
    abs_cos_x = abs(rotation_im[0,0]) 
    abs_sin_x = abs(rotation_im[0,1])
    abs_cos_y = abs(rotation_im[1,0]) 
    abs_sin_y = abs(rotation_im[1,1])

    # find the new width and height bounds
    bound_w = int(width * abs_cos_x +  height *  abs_sin_x )
    bound_h = int(height * abs_sin_y  + width * abs_cos_y )

    # subtract old image center (bringing image back to origo) and adding the new image center coordinates
    rotation_im[0, 2] += bound_w/2 - image_center[0] - image_center[1] * x_shear
    rotation_im[1, 2] += bound_h/2 - image_center[1] - image_center[0] * y_shear

    # board_mode = cv2.BORDER_CONSTANT
    # color = (random.randint(230,250), random.randint(230,250), random.randint(230,250))
    
    # # rotate image with the new bounds and translated rotation imrix
    # im = cv2.warpAffine(im, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=board_mode, borderValue=color)
    # Check if image has an alpha channel
    if im.shape[2] < 4:
        # If not, add one
        im = cv2.cvtColor(im, cv2.COLOR_BGR2BGRA)

    # Separate the color and alpha channels
    color = im[..., :3]
    alpha = im[..., 3]

    # Rotate the color channels
    color = cv2.warpAffine(color, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0))

    # Rotate the alpha channel
    alpha = cv2.warpAffine(alpha, rotation_im, (bound_w, bound_h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # Merge the color and alpha channels
    im = cv2.merge([color, alpha])

    # Define the original corner points
    corners = np.array([[0, 0], [width, 0], [0, height], [width, height]], dtype=np.float32)
    
    # Apply the transformation matrix to the corner points
    transformed_corners = cv2.transform(np.array([corners]), rotation_im)[0]
    print('transformed_corners',transformed_corners)
    #print(rotated_im.shape)
    return im, rotation_im, transformed_corners

def place_on_background(img, background):
    # # Resize image
    # size = random.uniform(*size_range)
    # img = cv2.resize(img, None, fx=size, fy=size)

    bk_height, bk_width = background.shape[:2]
    # scale the image
    img = scale_image(img, bk_height, bk_width)

    # Add random tilts
    # img = apply_perspective_transform(img, tilt_range)
    # img = rotate_and_shear_image(img, bk_height, bk_width)
    img, rotation_im, transformed_corners = rotate_and_shear_image(im=img)
    print('rotation_im',rotation_im)
    # img = stackoverflow_rotate_and_shear_image(img)

    # Ensure the document image is not larger than the background
    if img.shape[0] > background.shape[0] or img.shape[1] > background.shape[1]:
        print("Resized document image is larger than the background. Skipping...")
        return None, None, None

    # Get random position on background
    max_x = background.shape[1] - img.shape[1]
    max_y = background.shape[0] - img.shape[0]
    x = random.randint(0, max_x)
    y = random.randint(0, max_y)

    # Overlay the document image on the background
    for c in range(0, 3):
        background[y:y+img.shape[0], x:x+img.shape[1], c] = img[..., c] * (img[..., 3]/255.0) +  background[y:y+img.shape[0], x:x+img.shape[1], c] * (1.0 - img[..., 3]/255.0)

    # Return the final position and size of the image
    return background, (x, y, img.shape[1], img.shape[0]), transformed_corners


def create_mask(img, bbox, transformed_corners):
    # Initialize an empty mask
    mask = np.zeros_like(img[..., 0])

    # Draw a polygon (document boundary) on the mask
    x, y, w, h = bbox
    corners = np.array([[x + transformed_corners[0][0], y + transformed_corners[0][1]], 
                        [x+transformed_corners[1][0], y + transformed_corners[1][1]], 
                        [x + transformed_corners[3][0], y+transformed_corners[3][1]],
                        [x+transformed_corners[2][0], y+transformed_corners[2][1]]], dtype=np.int32)
    cv2.fillPoly(mask, [corners], 255)

    return mask

# Paths to folders
doc_folder = 'cropped_documents'
bg_folder = 'backgrounds'
mask_folder = 'masks'  # Folder to save masks

# Loop over all document images
for doc_name in os.listdir(doc_folder):
    doc_path = os.path.join(doc_folder, doc_name)
    print(f"Reading document image from: {doc_path}")  # Debug print
    doc_img = cv2.imread(doc_path)
    if doc_img is None:
        print(f"Failed to read document image from: {doc_path}")
        continue

    # Loop over all background images
    for bg_name in os.listdir(bg_folder):
        bg_path = os.path.join(bg_folder, bg_name)
        print(f"Reading background image from: {bg_path}")  # Debug print
        bg_img = cv2.imread(bg_path)
        if bg_img is None:
            print(f"Failed to read background image from: {bg_path}")
            continue

        # Place document on background
        img, bbox, transformed_corners = place_on_background(doc_img, bg_img)

        # Check if image is not None before saving
        if img is not None:
            # Save image and annotation
            cv2.imwrite('dataset/{}_{}.png'.format(doc_name, bg_name), img)
            with open('annotations/{}_{}.txt'.format(doc_name, bg_name), 'w') as f:
                f.write(','.join(map(str, bbox)))
            # Create the mask image
            mask = create_mask(img, bbox, transformed_corners)
            # Save the mask image
            mask_path = os.path.join(mask_folder, '{}_{}.png'.format(doc_name, bg_name))
            cv2.imwrite(mask_path, mask)
        else:
            print(f"Skipping saving for {doc_name} on {bg_name} due to size mismatch.")


Reading document image from: cropped_documents\001_jpg.rf.a831d3c36a427ca0073fbad72a1a6d3c.jpg
Reading background image from: backgrounds\-1x-1.jpg
transformed_corners [[-1.7749730e-01  2.4213783e+02]
 [ 2.6716074e+02 -4.3569946e-01]
 [ 1.2683929e+02  4.2243570e+02]
 [ 3.9417752e+02  1.7986217e+02]]
rotation_im [[ 7.77145961e-01  5.47486137e-01 -1.77497303e-01]
 [-7.05155583e-01  7.77145961e-01  2.42137829e+02]]
Reading background image from: backgrounds\0731617fa7e5d174286cf5594ba67fba.jpg
transformed_corners [[-4.6583693e-02  1.7139967e+02]
 [ 1.6077230e+01 -2.9414368e-01]
 [ 1.3392276e+02  1.8229414e+02]
 [ 1.5004657e+02  1.0600327e+01]]
rotation_im [[ 8.71557427e-02  1.07175484e+00 -4.65836935e-02]
 [-9.28074682e-01  8.71557427e-02  1.71399674e+02]]
Reading background image from: backgrounds\10-tips-to-create-a-perfect-workspace-at-home-2124.jpg
transformed_corners [[ 4.7244516e+02  3.3661552e+01]
 [ 4.2259207e+02  6.3706775e+02]
 [ 4.9407898e+01 -6.7722321e-02]
 [-4.4519043e-01  6

ValueError: not enough values to unpack (expected 3, got 2)

In [4]:
#### Create Negative Samples
def create_negative_mask(img):
    # Initialize an empty mask
    mask = np.zeros_like(img[..., 0])

    # Draw a polygon (document boundary) on the mask
    # x, y, w, h = bbox
    # corners = np.array([[x + transformed_corners[0][0], y + transformed_corners[0][1]], 
    #                     [x+transformed_corners[1][0], y + transformed_corners[1][1]], 
    #                     [x + transformed_corners[3][0], y+transformed_corners[3][1]],
    #                     [x+transformed_corners[2][0], y+transformed_corners[2][1]]], dtype=np.int32)
    # cv2.fillPoly(mask, [corners], 255)

    return mask

# Paths to folders
doc_folder = 'cropped_documents'
bg_folder = 'backgrounds'
mask_folder = 'negative_masks'  # Folder to save masks

# Loop over all background images
for bg_name in os.listdir(bg_folder):
    bg_path = os.path.join(bg_folder, bg_name)
    print(f"Reading background image from: {bg_path}")  # Debug print
    bg_img = cv2.imread(bg_path)
    if bg_img is None:
        print(f"Failed to read image from: {bg_path}")
    else:
        # Save image and annotation
        cv2.imwrite('negative_dataset/negative_{}.png'.format(bg_name), bg_img)
        # Create the mask image
        mask = create_negative_mask(bg_img)
        # Save the mask image
        mask_path = os.path.join(mask_folder, 'negative_{}.png'.format(bg_name))
        cv2.imwrite(mask_path, mask)

Reading background image from: backgrounds\-1x-1.jpg
Reading background image from: backgrounds\0731617fa7e5d174286cf5594ba67fba.jpg
Reading background image from: backgrounds\10-tips-to-create-a-perfect-workspace-at-home-2124.jpg
Reading background image from: backgrounds\1000_F_246987288_0CNI5GyhHWvSftaO1igXaIfNztJcoVcf.jpg
Reading background image from: backgrounds\2018_city_park_boardwalk_summer_hintringer.jpg
Reading background image from: backgrounds\750x500-elwha-river-credit-john-mcmillan.jpg
Reading background image from: backgrounds\90913081_m-1.jpg
Reading background image from: backgrounds\aerial-view-business-data-analysis-graph_53876-119697.jpg
Reading background image from: backgrounds\asher-mitilinakis-desk-setup-01-1.jpg
Reading background image from: backgrounds\blanket-bed-covering-insulate-keep-warm_1061358-61603.jpg
Reading background image from: backgrounds\Blog-feature-image-7-Tips-for-How-to-Use-Google-Workspace-Like-a-Pro-1024x681.jpg
Reading background image f

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:786: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
